In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
# df_hist = utils.read_df_pkl('../input/hist_clean*')
# df_new = utils.read_df_pkl('../input/new_clean*')
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mercha*0*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00,  4.60it/s]


SystemExit: 

In [24]:
for df in [df_hist, df_new]:
#     df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
#     df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
#     df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
    df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
    df['yyyymmdd'] = df['purchase_date'].map(lambda x: str(x)[:10])
# df_hist = df_hist.set_index(key).join(train_test['first_active_month']).reset_index()
# df_new = df_new.set_index(key).join(train_test['first_active_month']).reset_index()

In [20]:
from time_utils import date_add_days

hist_max_date = df_hist.groupby(key)['purchase_date'].max()
hist_max_date.name = 'hist_purchase_date_max'
hist_max_date = hist_max_date.to_frame()
# month_maxは翌月の数字にする
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
hist_max_date['hist_purchase_month_max'] = hist_max_date['hist_purchase_month_max'].map(lambda x: str(x)[:6])

hist_min_date = df_hist.groupby(key)['purchase_date'].min()
hist_min_date.name = 'hist_purchase_date_min'
hist_min_date = hist_min_date.to_frame()
hist_min_date['hist_purchase_month_min'] = hist_min_date['hist_purchase_date_min'].map(lambda x: str(x)[:7])


new_max_date = df_new.groupby(key)['purchase_date'].max()
new_max_date.name = 'new_purchase_date_max'
new_max_date = new_max_date.to_frame()
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_date_max'].map(lambda x: date_add_days(x, 32) if int(str(x)[8:10])<=15 else date_add_days(x, 20))
new_max_date['new_purchase_month_max'] = new_max_date['new_purchase_month_max'].map(lambda x: str(x)[:6])

new_min_date = df_new.groupby(key)['purchase_date'].min()
new_min_date.name = 'new_purchase_date_min'
new_min_date = new_min_date.to_frame()

# month_maxは翌月の数字にする
new_min_date['new_purchase_month_min'] = new_min_date['new_purchase_date_min'].map(lambda x: str(x)[:7])

df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
df_hist = df_hist.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)
df_new = df_new.join(hist_max_date).join(hist_min_date).join(new_max_date).join(new_min_date)

df_hist['hist_purchase_month_max'] = df_hist['hist_purchase_month_max'].map(lambda x: str(x)[:4] + '-' + str(x)[4:])
df_hist['new_purchase_month_max'] = df_hist['new_purchase_month_max'].map(lambda x: str(x)[:4] + '-' + str(x)[4:])
df_hist['new_purchase_month_max'] = df_hist['new_purchase_month_max'].replace('nan-', np.nan)

df_hist['hist_purchase_month_max'] = pd.to_datetime(df_hist['hist_purchase_month_max'])
df_hist['hist_purchase_month_min'] = pd.to_datetime(df_hist['hist_purchase_month_min'])
df_hist['new_purchase_month_max'] = pd.to_datetime(df_hist['new_purchase_month_max'])
df_hist['new_purchase_month_min'] = pd.to_datetime(df_hist['new_purchase_month_min'])

df_new['hist_purchase_month_max'] = df_new['hist_purchase_month_max'].map(lambda x: str(x)[:4] + '-' + str(x)[4:])
df_new['new_purchase_month_max'] = df_new['new_purchase_month_max'].map(lambda x: str(x)[:4] + '-' + str(x)[4:])
df_new['new_purchase_month_max'] = df_new['new_purchase_month_max'].replace('nan-', np.nan)

df_new['new_purchase_month_max'] = pd.to_datetime(df_new['new_purchase_month_max'])
df_new['new_purchase_month_min'] = pd.to_datetime(df_new['new_purchase_month_min'])
df_new['hist_purchase_month_max'] = pd.to_datetime(df_new['hist_purchase_month_max'])
df_new['hist_purchase_month_min'] = pd.to_datetime(df_new['hist_purchase_month_min'])

In [35]:
df_new_lag1 = df_new[df_new['month_lag']==1]
df_new_lag2 = df_new[df_new['month_lag']==2]

df = df_hist

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]
print(auth1.shape)
print(auth0.shape)

cat1_0 = False
# cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
    new_cat1 = df_new[df_new.category_1==1]
    new_cat0 = df_new[df_new.category_1==0]
#     del auth1, auth0
#     gc.collect()

auth1_lag0 = auth1[auth1['month_lag']==0]
auth1_lag1 = auth1[auth1['month_lag']==-1]
auth1_lag2 = auth1[auth1['month_lag']==-2]
auth1_lag02 = auth1[auth1['month_lag']>=-2]
auth1_lag05 = auth1[auth1['month_lag']>=-5]

auth0_lag0 = auth0[auth0['month_lag']==0]
auth0_lag1 = auth0[auth0['month_lag']==-1]
auth0_lag2 = auth0[auth0['month_lag']==-2]
auth0_lag02 = auth0[auth0['month_lag']>=-2]
auth0_lag05 = auth0[auth0['month_lag']>=-5]

(26595452, 23)
(2516909, 23)


In [39]:
prefix = '203_pst'
prefix = '303_pst'
prefix = '403_pst'
new_df_list = [df_new, df_new_lag1, df_new_lag2]
new_fname_list = ['new', 'new_lag1', 'new_lag2']

debug = False
if debug:
    train_test = train_test.head(10000)
        

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


#========================================================================
# 集計が必要なFeature
#========================================================================
for df, fname in zip(tqdm(new_df_list), new_fname_list):
    if debug:
        df = df.head(3000)
#     df.drop(['level_0', 'index'], axis=1, inplace=True)
#     sys.exit()
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    
    df['diff_date_from_new_min_month'] = (df['new_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_new_max_month'] = (df['new_purchase_month_max'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_min_month'] = (df['hist_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_max_month'] = (df['hist_purchase_month_max'] - df['purchase_date']).dt.days

    aggs = {}
    aggs['diff_date_from_new_min_month'] = ['mean', 'max', 'min', 'std' ]
    aggs['diff_date_from_new_max_month'] = ['mean', 'max', 'min', 'std' ]
    aggs['diff_date_from_hist_min_month'] = ['mean', 'max', 'min', 'std']
    aggs['diff_date_from_hist_max_month'] = ['mean', 'max', 'min', 'std']
#     aggs['diff_date_from_new_min_month'] = ['skew']
#     aggs['diff_date_from_new_max_month'] = ['skew']
#     aggs['diff_date_from_hist_min_month'] = ['skew']
#     aggs['diff_date_from_hist_max_month'] = ['skew']

#     col_unique =['subsector_id', 'merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    col_unique =['yyyymmdd']
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    
    
#     df = pd.get_dummies(df, columns=['category_2', 'category_3'])
#     aggs = {
#     'category_1': ['sum', 'mean'],
#     'category_2_1.0': ['mean'],
#     'category_2_2.0': ['mean'],
#     'category_2_3.0': ['mean'],
#     'category_2_4.0': ['mean'],
#     'category_2_5.0': ['mean'],
#     'category_3_A': ['mean'],
#     'category_3_B': ['mean'],
#     'category_3_C': ['mean'],
#     }
    
#     for col in col_unique:
#         aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum', 'max','mean','var']
#     aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean','var']
    aggs['card_id'] = ['count']
    
    if ('level_0' and 'index' not in df.columns) and key not in df.columns:
        df.reset_index(inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    # add feature
#     df_agg[f'{fname}_amount_sum_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
#     df_agg[f'{fname}_amount_mean_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    
    #========================================================================
    # monthly agg
    #========================================================================
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    
#     train_test = train_test.join(df_agg)
    train_test = train_test.join(df_agg).join(month_agg)
    print(train_test.shape)
    
    elo_save_feature(prefix, train_test, feat_check=False)
print('Complete!!')



  0%|          | 0/3 [00:00<?, ?it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 100.98it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 172.30it/s]

Aggregation Start!
(1963031, 31)
(325540, 61)




 33%|███▎      | 1/3 [00:28<00:56, 28.44s/it]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 102.61it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 176.45it/s]

Aggregation Start!
(1027617, 31)
(325540, 61)




 67%|██████▋   | 2/3 [00:54<00:27, 27.86s/it]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 92.28it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 167.42it/s]

Aggregation Start!
(935414, 31)
(325540, 61)




100%|██████████| 3/3 [01:21<00:00, 27.37s/it]



In [40]:
prefix = '203_pst'
prefix = '303_pst'
prefix = '403_pst'
hist_df_list = [auth1, auth1_lag0, auth1_lag1, auth1_lag2, auth1_lag02, auth1_lag05, auth0, auth0_lag0, auth0_lag1, auth0_lag2, auth0_lag02, auth0_lag05] 
hist_fname_list = ['auth1', 'auth1_lag0', 'auth1_lag1', 'auth1_lag2', 'auth1_lag02', 'auth1_lag05', 'auth0', 'auth0_lag0', 'auth0_lag1', 'auth0_lag2', 'auth0_lag02', 'auth0_lag05']

for df, fname in zip(tqdm(hist_df_list), hist_fname_list):
    if debug:
        df = df.head(3000)
    
    df_train = utils.read_df_pkl('../input/train0*')
    df_test = utils.read_df_pkl('../input/test0*')
    df_train.set_index(key, inplace=True)
    df_test.set_index(key, inplace=True)
    train_test = pd.concat([df_train, df_test], axis=0)
    
    # new
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    
    df['diff_date_from_new_min_month'] = (df['new_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_new_max_month'] = (df['new_purchase_month_max'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_min_month'] = (df['hist_purchase_month_min'] - df['purchase_date']).dt.days
    df['diff_date_from_hist_max_month'] = (df['hist_purchase_month_max'] - df['purchase_date']).dt.days

    aggs = {}
    aggs['diff_date_from_new_min_month'] = ['mean', 'max', 'min', 'std' ]
    aggs['diff_date_from_new_max_month'] = ['mean', 'max', 'min', 'std' ]
    aggs['diff_date_from_hist_min_month'] = ['mean', 'max', 'min', 'std']
    aggs['diff_date_from_hist_max_month'] = ['mean', 'max', 'min', 'std']
#     aggs['diff_date_from_new_min_month'] = ['skew']
#     aggs['diff_date_from_new_max_month'] = ['skew']
#     aggs['diff_date_from_hist_min_month'] = ['skew']
#     aggs['diff_date_from_hist_max_month'] = ['skew']

#     col_unique =['subsector_id', 'merchant_id', 'merchant_category_id', 'yyyymmdd', 'yyyy_week']
    col_unique =['yyyymmdd']
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    
    
#     df = pd.get_dummies(df, columns=['category_2', 'category_3'])
#     aggs = {
#     'category_1': ['sum', 'mean'],
#     'category_2_1.0': ['mean'],
#     'category_2_2.0': ['mean'],
#     'category_2_3.0': ['mean'],
#     'category_2_4.0': ['mean'],
#     'category_2_5.0': ['mean'],
#     'category_3_A': ['mean'],
#     'category_3_B': ['mean'],
#     'category_3_C': ['mean'],
#     }
    
#     for col in col_unique:
#         aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum', 'max','mean','var']
#     aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean','var']
    aggs['card_id'] = ['count']
    
    if ('level_0' and 'index' not in df.columns) and key not in df.columns:
        df.reset_index(inplace=True)
    
    new_columns = get_new_columns(fname, aggs)
    
    print('Aggregation Start!')
    print(df.shape)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    
    # add feature
#     df_agg[f'{fname}_amount_sum_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
#     df_agg[f'{fname}_amount_mean_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    
    #========================================================================
    # monthly agg
    #========================================================================
    new_columns = get_new_columns(fname + '_monthly_avg', aggs)
    month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
        
    month_agg.columns = new_columns
    month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
    
#     train_test = train_test.join(df_agg)
    train_test = train_test.join(df_agg).join(month_agg)
    print(train_test.shape)
    
    elo_save_feature(prefix, train_test, feat_check=False)
print('Complete!!')



  0%|          | 0/12 [00:00<?, ?it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 102.79it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 172.40it/s]

Aggregation Start!
(26595452, 29)
(325540, 61)




  8%|▊         | 1/12 [00:52<09:36, 52.45s/it]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 96.66it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 167.24it/s]

Aggregation Start!
(3178354, 29)
(325540, 61)




 17%|█▋        | 2/12 [01:20<07:31, 45.17s/it]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 98.55it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 181.85it/s]

...

Aggregation Start!
(927323, 29)
(325540, 61)




 92%|█████████▏| 11/12 [05:37<00:26, 26.32s/it]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 97.20it/s]


  0%|          | 0/3 [00:00<?, ?it/s]


100%|██████████| 3/3 [00:00<00:00, 171.08it/s]

Aggregation Start!
(1617002, 29)
(325540, 61)




100%|██████████| 12/12 [06:04<00:00, 26.68s/it]



Complete!!


In [5]:
prefix = '303_pst'
prefix = '403_pst'

import glob
# #========================================================================
# # これは作成済のFeatureを読み込んで計算する
# #========================================================================
combi_list = [ ['new', 'new_lag1'] ,['new', 'auth1'] ,['new', 'auth1_lag0'] ,['new', 'auth1_lag02'] ,['new', 'auth1_lag05'] ,['new', 'auth0_lag0'] ,
['new', 'auth0_lag02'] ,['new', 'auth0_lag05'] ,['auth1_lag0', 'auth1_lag02'] ,['auth1_lag0', 'auth1_lag05'] ,['auth1_lag02', 'auth1_lag05'] ,['auth0_lag02', 'auth0_lag05']
]

comp_cols = [
    'amount_sum_per_installments_sum'
    ,'amount_mean_per_installments_mean'
    ,'purchase_amount_sum' 
    ,'purchase_amount_mean' 
    ,'purchase_amount_max' 
    ,'purchase_amount_min' 
    ,'installments_max' 
    ,'installments_var' 
    ,'category_1_sum' 
    ,'category_1_mean' 
    ,'month_lag_mean' 
    ,'month_lag_var' 
    ,'month_lag_skew' 
    ,'month_diff_mean' 
    ,'month_diff_var' 
    ,'month_diff_skew' 
    ,'card_id_size' 
    ,'yyyymmdd_nunique'
    ,'subsector_id_nunique'
    ,'merchant_id_nunique'
    ,'merchant_category_id_nunique'
     'diff_date_from_new_min_month_mean'
     ,'diff_date_from_hist_max_month_mean'
     ,'diff_date_from_new_min_month_max'
     ,'diff_date_from_hist_max_month_max'
     ,'diff_date_from_new_min_month_min'
     ,'diff_date_from_hist_min_month_min'
]

tmp_feature_list = glob.glob(f'../features/1_first_valid/{prefix}*.gz')
# tmp_feature_list = glob.glob(f'../features/8_ensemble/{prefix}*.gz') + glob.glob(f'../features/4_winner/{prefix}*.gz')
feature_list = []
for f in tmp_feature_list:
    if f.count('pst_ratio_') or f.count('pst_diff_'):continue
    for col in comp_cols:
        if f.count(col):
            feature_list.append(f)
            
base = utils.read_df_pkl('../input/base_first*0*')
p_list = utils.parallel_load_data(path_list=feature_list)
df_feat = pd.concat(p_list, axis=1)
train_test = pd.concat([base[key], df_feat], axis=1)

for (fm1, fm2) in tqdm(combi_list):
    for col in comp_cols:
#         203_pst_auth0_lag02_monthly_avg_purchase_amount_min
        try:
            train_test[f"ratio_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] / train_test[f"{prefix}_{fm2}_{col}@"]
            train_test[f"diff_{fm1}_{fm2}_{col}"] = train_test[f"{prefix}_{fm1}_{col}@"] - train_test[f"{prefix}_{fm2}_{col}@"]
        except KeyError:
#             print(fm1, fm2, col)
            continue
        
print(train_test.shape)

ratio_diff_cols = [col for col in train_test.columns if col[:5]=='ratio' or col[:4]=='diff']

elo_save_feature(prefix, train_test[ratio_diff_cols], feat_check=False)

100%|██████████| 12/12 [00:02<00:00,  4.48it/s]


(325540, 703)
